In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\eweber\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
news_url = 'https://redplanetscience.com/'
browser.visit(news_url)

In [8]:
news_html = browser.html
soup = bs(news_html, 'html.parser')

In [9]:
#News Title
news_title = soup.find('div', class_='content_title').text
print(news_title)

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover


In [10]:
#News Paragraph
news_p = soup.find('div', class_='article_teaser_body').text
print(news_p)

NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 


In [11]:
#Photo
pic_url='https://spaceimages-mars.com'
browser.visit(pic_url)

In [12]:
browser.links.find_by_partial_text("FULL IMAGE").click()

In [13]:
pic_html = browser.html
soup = bs(pic_html, 'html.parser')
pic_url = soup.find('img', class_ = 'fancybox-image')['src']

In [253]:
featured_image_url = pic_html + pic_url
featured_image_url

'<html class="fancybox-margin fancybox-lock"><head>\n      <meta charset="utf-8">\n      <meta name="viewport" content="width=device-width, initial-scale=1">\n      <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css">\n      <!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->\n      <link rel="stylesheet" type="text/css" href="css/app.css">\n      <link rel="stylesheet" type="text/css" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">\n      <title>Space Image</title>\n   <style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>\n   <body>\n      <div class="header">\n         <nav class="navbar navbar-expand-lg ">\n            <a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"><span class="logo">Jet Propulsion Laboratory</span>\n            <span class="logo1">California Institute of Technology</span></a>\n            <button class="navbar-toggler" t

In [254]:
#Facts
facts_url = 'https://galaxyfacts-mars.com'
browser.visit(facts_url)

In [255]:
facts_html = browser.html
soup = bs(facts_html, 'html.parser')
#mars_facts = soup.find('table', class_ = 'tbody')

In [256]:
mars_table = pd.read_html(facts_html)
mars_table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [257]:
mars_df = mars_table[0]
mars_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [258]:
mars_df.columns = ["Categories", "Mars", "Earth"]
mars_df

,Categories,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [264]:
marshtml = (mars_df.to_html())

In [265]:
print(marshtml)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Categories</th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Mars - Earth Comparison</td>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Temperature:</td>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
   

In [267]:
mars_df.to_csv('../marsData.csv',index=False)

In [260]:
#Hemisphere
hemi_url = "https://marshemispheres.com/"
browser.visit(hemi_url)

In [261]:
hemi_html = browser.html
soup = bs(hemi_html, "html.parser")

In [262]:
hemi = soup.find_all("div", class_="item")
#hemi

In [279]:
hemi_info = []
for result in hemi:
    h_title = result.find("h3").text
    hemi_img = x.find("a", class_="itemLink product-item")["href"]
    
    browser.visit(hemi_url + hemi_img)
    
    image_html = browser.html
    web_info = bs(image_html, "html.parser")
    
    #browser.links.find_by_partial_text('Next').click()
    try:
        
        img_url = hemi_url + web_info.find("img", class_="wide-image").link
        browser.links.find_by_partial_text('Next').click()
    except:
        ""
    
    hemi_info.append({"title":h_title, "img_url":img_url})
    
    print(h_title)
    print(img_url)

Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
